In [2]:
import torch
import os, sys

project_root = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir, os.pardir)
)

stage1_root = os.path.join(project_root, "stage1")
sys.path.insert(0, stage1_root)

# now 'src' is a top-level package
from src.gpt2small import GPTModel, GPTConfig124


encoded: [32, 582, 1297, 502]
encoded_tensor: torch.Size([1, 4])
output: tensor([[   32,   582,  1297,   502, 24106, 40901, 46947, 45439, 41121, 10983,
         37408, 33219, 13597,   384, 41274, 27228, 32767,   972, 16051,  1462,
          6565, 26531, 19694, 30470]])
output length: 24
A man told me accumulation thumbnail Flask 406 propensity Hat lush Tulsolk se pandemonium 179 Hutement acresto emptyDa ISOetsu


In [3]:
cfg_pt = GPTConfig124(vocab_size=50257, context_length=256, emb_dim=768,
                   n_heads=12, n_layers=12, dropout=0.1, qkv_bias=False)